In [59]:
import pandas as pd
import numpy as np
import re

In [60]:
raw_df = pd.read_csv('./test_datasets/house_prices.csv')

In [61]:
def remove_duplicated_cols(dataset):
    # .loc - selects data by labels
    # .loc - first argument(':' <- select all rows) second argument(select columns)
    # '~' sign flip the boolean value (ex: [False, False, True] -> [True, True, False])
    df = dataset.loc[:, ~dataset.columns.duplicated()]

    return df

# ---------------------------------------------------------------------------------
# 02) REMOVE COLUMNS WITH ALL EMPTY VALUES OR SAME VALUE IN EVERY ROW
# ---------------------------------------------------------------------------------
def remove_empty_or_constant_cols(dataset):
    # Remove columns with all NaN values
    df = dataset.dropna(axis=1, how='all')

    # Keep columns that has no same values for all the rows
    df = dataset.loc[:, dataset.nunique() > 1]

    return df

In [62]:
remove_duplicated_cols(raw_df)
remove_empty_or_constant_cols(raw_df)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,price.1,name
0,5/2/2014 0:00,3.130000e+05,3,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,NaN,Michele
1,5/2/2014 0:00,2.384000e+06,5,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,dsdfssdfs,Sharon
2,5/2/2014 0:00,3.420000e+05,3,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,NaN,NaN
3,5/2/2014 0:00,4.200000e+05,3,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,NaN,NaN
4,5/2/2014 0:00,5.500000e+05,4,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,7/9/2014 0:00,3.081667e+05,3,1.75,1510,6360,1.0,0,0,4,1510,0,1954,1979,501 N 143rd St,Seattle,WA 98133,NaN,NaN
4596,7/9/2014 0:00,5.343333e+05,3,2.50,1460,7573,2.0,0,0,3,1460,0,1983,2009,14855 SE 10th Pl,Bellevue,WA 98007,NaN,NaN
4597,7/9/2014 0:00,4.169042e+05,3,2.50,3010,7014,2.0,0,0,3,3010,0,2009,0,759 Ilwaco Pl NE,Renton,WA 98059,NaN,NaN
4598,7/10/2014 0:00,2.034000e+05,4,2.00,2090,6630,1.0,0,0,3,1070,1020,1974,0,5148 S Creston St,Seattle,WA 98178,NaN,NaN


In [63]:
raw_df

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,yr_renovated,street,city,statezip,country,price.1,test_col,name,Address,Phone Number
0,5/2/2014 0:00,3.130000e+05,3,1.50,1340,7912,1.5,0,0,3,...,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA,NaN,NaN,Michele,NaN,752655918.0
1,5/2/2014 0:00,2.384000e+06,5,2.50,3650,9050,2.0,0,4,5,...,0,709 W Blaine St,Seattle,WA 98119,USA,dsdfssdfs,NaN,Sharon,"Edirisinghe Stores Bare, CP Pura, Minneriya",NaN
2,5/2/2014 0:00,3.420000e+05,3,2.00,1930,11947,1.0,0,0,4,...,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA,NaN,NaN,NaN,NaN,NaN
3,5/2/2014 0:00,4.200000e+05,3,2.25,2000,8030,1.0,0,0,4,...,0,857 170th Pl NE,Bellevue,WA 98008,USA,NaN,NaN,NaN,NaN,NaN
4,5/2/2014 0:00,5.500000e+05,4,2.50,1940,10500,1.0,0,0,4,...,1992,9105 170th Ave NE,Redmond,WA 98052,USA,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,7/9/2014 0:00,3.081667e+05,3,1.75,1510,6360,1.0,0,0,4,...,1979,501 N 143rd St,Seattle,WA 98133,USA,NaN,NaN,NaN,NaN,NaN
4596,7/9/2014 0:00,5.343333e+05,3,2.50,1460,7573,2.0,0,0,3,...,2009,14855 SE 10th Pl,Bellevue,WA 98007,USA,NaN,NaN,NaN,NaN,NaN
4597,7/9/2014 0:00,4.169042e+05,3,2.50,3010,7014,2.0,0,0,3,...,0,759 Ilwaco Pl NE,Renton,WA 98059,USA,NaN,NaN,NaN,NaN,NaN
4598,7/10/2014 0:00,2.034000e+05,4,2.00,2090,6630,1.0,0,0,3,...,0,5148 S Creston St,Seattle,WA 98178,USA,NaN,NaN,NaN,NaN,NaN


In [64]:
raw_df.isna().mean()

date             0.000000
price            0.000000
bedrooms         0.000000
bathrooms        0.000000
sqft_living      0.000000
sqft_lot         0.000000
floors           0.000000
waterfront       0.000000
view             0.000000
condition        0.000000
sqft_above       0.000000
sqft_basement    0.000000
yr_built         0.000000
yr_renovated     0.000000
street           0.000000
city             0.000000
statezip         0.000000
country          0.000000
price.1          0.999130
test_col         1.000000
name             0.999348
Address          0.999783
Phone Number     0.999783
dtype: float64

In [67]:
def remove_sensitive_columns(df, threshold=0.7):
    """
    Automatically detect and remove columns containing sensitive/personal information:
    - Phone numbers
    - Emails
    - Postal codes / ZIP codes
    - Names / mostly text
    - Addresses
    
    Combines column name keywords and content detection.
    
    threshold: fraction of rows matching pattern to classify as sensitive
    """
    df = df.copy()
    cols_to_drop = []

    # --- Keywords for column names ---
    phone_keywords = ['phone', 'mobile', 'cell']
    email_keywords = ['email', 'mail']
    postal_keywords = ['zip', 'postal', 'postcode']
    name_keywords = ['name', 'fullname', 'first', 'last']
    address_keywords_cols = ['address', 'addr', 'street', 'location', 'home', 'billing', 'mailing']
    address_keywords_pattern = r"(?:St|Ave|Rd|Lane|Blvd|Drive|Court|Way|Pl|Sq|Circle|Terrace|Parkway)"

    # --- Patterns for content detection ---
    phone_pattern = r'^\+?\d{10,15}$'  # e.g., 0712345678 or +94712345678
    email_pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    postal_pattern = r'^[A-Z]{0,2}\s?\d{5}(-\d{4})?$'
    address_pattern = rf".*\d+.*{address_keywords_pattern}.*"  # number + street keyword

    for col in df.columns:
        series = df[col].astype(str)
        col_lower = col.lower()

        # --- Column name checks ---
        phone_name_flag = any(k in col_lower for k in phone_keywords)
        email_name_flag = any(k in col_lower for k in email_keywords)
        postal_name_flag = any(k in col_lower for k in postal_keywords)
        name_name_flag = any(k in col_lower for k in name_keywords)
        address_name_flag = any(k in col_lower for k in address_keywords_cols)

        # --- Content-based checks ---
        phone_ratio = series.str.match(phone_pattern).mean()
        email_ratio = series.str.match(email_pattern).mean()
        postal_ratio = series.str.match(postal_pattern).mean()
        address_ratio = series.str.contains(address_pattern, flags=re.IGNORECASE, regex=True).mean()
        text_ratio = (~series.str.replace(r'\s','').str.isnumeric()).mean()

        # --- Decision rules ---
        if phone_ratio >= threshold or phone_name_flag:
            cols_to_drop.append(col)
        elif email_ratio >= threshold or email_name_flag:
            cols_to_drop.append(col)
        elif postal_ratio >= threshold or postal_name_flag:
            cols_to_drop.append(col)
        elif (address_ratio >= 0.1 or address_name_flag):  # small fraction content + name signal
            cols_to_drop.append(col)
        elif text_ratio >= 0.8 and name_name_flag:  # mostly text + name keyword
            cols_to_drop.append(col)

    # Remove duplicates and drop columns
    cols_to_drop = list(set(cols_to_drop))
    if cols_to_drop:
        print(f"Dropping columns detected as sensitive/personal info: {cols_to_drop}")
        df = df.drop(columns=cols_to_drop)

    return df

In [68]:
remove_sensitive_columns(raw_df)

Dropping columns detected as sensitive/personal info: ['name', 'statezip', 'street', 'Address', 'Phone Number']


,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,city,country,price.1,test_col
0,5/2/2014 0:00,3.130000e+05,3,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,Shoreline,USA,NaN,NaN
1,5/2/2014 0:00,2.384000e+06,5,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,Seattle,USA,dsdfssdfs,NaN
2,5/2/2014 0:00,3.420000e+05,3,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,Kent,USA,NaN,NaN
3,5/2/2014 0:00,4.200000e+05,3,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,Bellevue,USA,NaN,NaN
4,5/2/2014 0:00,5.500000e+05,4,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,Redmond,USA,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,7/9/2014 0:00,3.081667e+05,3,1.75,1510,6360,1.0,0,0,4,1510,0,1954,1979,Seattle,USA,NaN,NaN
4596,7/9/2014 0:00,5.343333e+05,3,2.50,1460,7573,2.0,0,0,3,1460,0,1983,2009,Bellevue,USA,NaN,NaN
4597,7/9/2014 0:00,4.169042e+05,3,2.50,3010,7014,2.0,0,0,3,3010,0,2009,0,Renton,USA,NaN,NaN
4598,7/10/2014 0:00,2.034000e+05,4,2.00,2090,6630,1.0,0,0,3,1070,1020,1974,0,Seattle,USA,NaN,NaN
